In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
import seaborn as sns

import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

from itertools import combinations
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings(action='ignore')
import math

In [5]:
boston = load_boston()
boston_df = pd.DataFrame(boston['data'])
# for i in range(13):
#     boston_df.rename(columns = {i: boston['feature_names'][i]}, inplace=True)
boston_df['PRICE'] = boston['target']

In [6]:
# def reset_parameters(self) -> None:
#     init.kaiming_uniform_(self.weight, a=math.sqrt(5))
#     if self.bias is not None:
#         fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
#         bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
#         init.uniform_(self.bias, -bound, bound)

In [7]:
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        print(self.parameters())
        self.initialize_weights()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 80),
            nn.ReLU(),
            #nn.Dropout(0.05),
            nn.Linear(80, 160),
            nn.ReLU(),
            #nn.Dropout(0.05),
            nn.Linear(160, 80),
            nn.ReLU(),
            #nn.Dropout(0.05),
            nn.Linear(80, output_size)
        )
        
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
    
    def initialize_weights(self):
        # track all layers
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_uniform_(m.weight)

                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

            elif isinstance(m, nn.Linear):
                nn.init.kaiming_uniform_(m.weight)
                nn.init.constant_(m.bias, 0)


In [8]:
import pandas as pd
from sklearn.metrics import r2_score

def calculate_r2(x,y=[]):
    X = Variable(torch.FloatTensor(x))
    result = model(X)
    result = result.data[:,0].numpy()
    
    if len(y) != 0:
        r2=r2_score(result, y)
        print("R-Squared", r2)
        return r2
    else:
        print("returning predictions")
        return result

In [9]:
# model = Model(12, 1)
# optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
# loss_fn = nn.MSELoss(reduction='sum')

In [10]:
# iter = combinations(range(10), 5)
# for i in iter:
#     print(i)
y = boston_df['PRICE']
iter_result = []
iter_list = combinations(range(13), 1)
iter_len = 0
X_list = [0 for i in range(13)]
X_train = [0 for i in range(13)]
X_test = [0 for i in range(13)]
y_train = [0 for i in range(13)]
y_test = [0 for i in range(13)]

for i in iter_list:
    X = boston_df.iloc[:, :13]
    for n in i:
        X.drop(columns=n, inplace=True)
    X_list[iter_len] = X
    X_train[iter_len], X_test[iter_len], y_train[iter_len], y_test[iter_len] = train_test_split(X_list[iter_len], y, test_size=0.2, shuffle=True, random_state=0)
    iter_len += 1

In [12]:
from sklearn.utils import shuffle
from torch.autograd import Variable
from collections import defaultdict

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Executing the model on:", device)

batch_size = 50
num_epochs = 300
# 여기 항상 바꿔야 함
batch_no = 404//batch_size
running_loss = 0
result1_list = []
result2_list = []
point1_dict = defaultdict(int)
point2_dict = defaultdict(int)


y = boston_df['PRICE']
iter_result = []
iter_list = combinations(range(13), 1)
iter_len = 0
X_list = [0 for i in range(13)]
X_train = [0 for i in range(13)]
X_test = [0 for i in range(13)]
y_train = [0 for i in range(13)]
y_test = [0 for i in range(13)]

for i in iter_list:
    X = boston_df.iloc[:, :13]
    for n in i:
        X.drop(columns=n, inplace=True)
    X_list[iter_len] = X
    X_train[iter_len], X_test[iter_len], y_train[iter_len], y_test[iter_len] = train_test_split(X_list[iter_len], y, test_size=0.2, shuffle=True, random_state=0)
    iter_len += 1



for _ in range(10):
    for i in range(iter_len):
        
        y = boston_df['PRICE']
        iter_result = []
        iter_list = combinations(range(13), 1)
        iter_len = 0
        X_list = [0 for i in range(13)]
        X_train = [0 for i in range(13)]
        X_test = [0 for i in range(13)]
        y_train = [0 for i in range(13)]
        y_test = [0 for i in range(13)]

        for j in iter_list:
            X = boston_df.iloc[:, :13]
            for n in j:
                X.drop(columns=n, inplace=True)
            X_list[iter_len] = X
            X_train[iter_len], X_test[iter_len], y_train[iter_len], y_test[iter_len] = train_test_split(X_list[iter_len], y, test_size=0.2, shuffle=True, random_state=0)
            iter_len += 1
        
        
        X_train[i] = X_train[i].values
        y_train[i] = y_train[i].values
        X_test[i] = X_test[i].values
        y_test[i] = y_test[i].values
        model = Model(12, 1)
        optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
        loss_fn = nn.MSELoss(reduction='sum')
    
        for epoch in range(num_epochs):
            X_train[i], y_train[i] = shuffle(X_train[i], y_train[i])
            
            for b in range(batch_no):
                start = b * batch_size
                end = start + batch_size
                inputs = Variable(torch.FloatTensor(X_train[i][start:end]))
                labels = Variable(torch.FloatTensor(y_train[i][start:end]))
                model.train()
                outputs = model(inputs)
                optimizer.zero_grad()
                
                loss = loss_fn(outputs, torch.unsqueeze(labels,dim=1))
                # 차원을 맞춰주는 것이 참 중요하다!
                
                loss.backward()
                optimizer.step()
                
                running_loss += loss.item()
            
        X=Variable(torch.FloatTensor(X_train[i]))
        result=model(X)
        pred=result.data[:,0].numpy()
        r2_score(pred,y_train[i])
        
        print(i, boston['feature_names'][i])
        model.train() 
        result1=calculate_r2(X_train[i],y_train[i])
    #     print(i, result1)
        model.eval()
        result2=calculate_r2(X_test[i],y_test[i])
        model.train() 
    #     print(i, result2)
        result1_list.append([result1, i, boston['feature_names'][i]])
        result2_list.append([result2, i, boston['feature_names'][i]])
        print('--------------------------------')
        
    result1_list.sort(key=lambda x:x[0])
    result2_list.sort(key=lambda x:x[0])
#     print("result1")
    for j, r in enumerate(result1_list):
        print(j)
        point1_dict[r[1]] += j
#     print("----------------------------------")
#     print("result2")
    for j, r in enumerate(result2_list):
        print(j)
        point2_dict[r[1]] += j
    
    result1_list = list()
    result2_list = list()
    
    print(point1_dict, point2_dict)

Executing the model on: cuda:0
<generator object Module.parameters at 0x0000011795FD2D60>
0 CRIM
R-Squared 0.9017045610705932
R-Squared 0.5199752578360779
--------------------------------
<generator object Module.parameters at 0x0000011796035190>
1 ZN
R-Squared 0.8697390941202994
R-Squared 0.3500449307086245
--------------------------------
<generator object Module.parameters at 0x0000011795FD2BA0>
2 INDUS
R-Squared 0.9028248448250122
R-Squared 0.35184284178166736
--------------------------------
<generator object Module.parameters at 0x0000011796035350>
3 CHAS
R-Squared 0.8435582853368634
R-Squared 0.4938098647463737
--------------------------------
<generator object Module.parameters at 0x0000011796035200>
4 NOX
R-Squared 0.8884692916918235
R-Squared 0.46188461099485334
--------------------------------
<generator object Module.parameters at 0x00000117960352E0>
5 RM
R-Squared 0.8919123537813193
R-Squared 0.5283613079260362
--------------------------------
<generator object Module.para

8 RAD
R-Squared 0.8593587823447455
R-Squared 0.49712796318163666
--------------------------------
<generator object Module.parameters at 0x0000011795FD2D60>
9 TAX
R-Squared 0.8900515231271515
R-Squared 0.5476351382979445
--------------------------------
<generator object Module.parameters at 0x0000011796035580>
10 PTRATIO
R-Squared 0.8950678913977651
R-Squared 0.5489101883179752
--------------------------------
<generator object Module.parameters at 0x0000011795FD2AC0>
11 B
R-Squared 0.9031943163455299
R-Squared 0.6547183540377324
--------------------------------
<generator object Module.parameters at 0x0000011796035C80>
12 LSTAT
R-Squared 0.6653165271935898
R-Squared -0.018204765069020024
--------------------------------
0
1
2
3
4
5
6
7
8
9
10
11
12
0
1
2
3
4
5
6
7
8
9
10
11
12
defaultdict(<class 'int'>, {12: 2, 8: 22, 7: 16, 3: 14, 1: 28, 6: 19, 4: 19, 5: 17, 0: 35, 2: 22, 10: 35, 9: 36, 11: 47}) defaultdict(<class 'int'>, {12: 3, 8: 18, 1: 21, 2: 13, 6: 30, 7: 37, 4: 13, 3: 17, 10: 

1 ZN
R-Squared 0.8414997236779228
R-Squared 0.3954426687045409
--------------------------------
<generator object Module.parameters at 0x00000117960353C0>
2 INDUS
R-Squared 0.9050992162490209
R-Squared 0.5043067955662593
--------------------------------
<generator object Module.parameters at 0x0000011796035CF0>
3 CHAS
R-Squared 0.8794494675401922
R-Squared 0.4121961683811928
--------------------------------
<generator object Module.parameters at 0x0000011796035A50>
4 NOX
R-Squared 0.9076530080330469
R-Squared 0.5249716449690409
--------------------------------
<generator object Module.parameters at 0x0000011796035900>
5 RM
R-Squared 0.892229018836986
R-Squared 0.39036517896763856
--------------------------------
<generator object Module.parameters at 0x0000011796035900>
6 AGE
R-Squared 0.8639488014702418
R-Squared 0.47123798489332336
--------------------------------
<generator object Module.parameters at 0x0000011796035580>
7 DIS
R-Squared 0.8316024179145759
R-Squared 0.631382399524264

In [ ]:
from sklearn.utils import shuffle
from torch.autograd import Variable

y = boston_df['PRICE']
X = boston_df.iloc[:, :13]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=0)

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Executing the model on:", device)

batch_size = 50
num_epochs = 1000
# 여기 항상 바꿔야 함
batch_no = 404//batch_size
running_loss = 0

X_train = X_train.values
y_train = y_train.values
X_test = X_test.values
y_test = y_test.values
model = Model(13, 1)
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
loss_fn = nn.MSELoss(reduction='sum')

for epoch in range(num_epochs):
    X_train, y_train = shuffle(X_train, y_train)
        
    for b in range(batch_no):
        start = b * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(X_train[start:end]))
        labels = Variable(torch.FloatTensor(y_train[start:end]))
        model.train()
        outputs = model(inputs)
        optimizer.zero_grad()
        
        loss = loss_fn(outputs, torch.unsqueeze(labels,dim=1))
        # 차원을 맞춰주는 것이 참 중요하다!
            
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
X=Variable(torch.FloatTensor(X_train))
result=model(X)
pred=result.data[:,0].numpy()
r2_score(pred,y_train)
model.train()
result1=calculate_r2(X_train,y_train)
model.eval()
result2=calculate_r2(X_test,y_test)
print('--------------------------------')

In [ ]:
# import pandas as pd
# from sklearn.metrics import r2_score

# X=Variable(torch.FloatTensor(X_train))
# result=model(X)
# pred=result.data[:,0].numpy()
# print(len(pred), len(y_train))
# r2_score(pred,y_train)

In [54]:
# result1=calculate_r2(X_train,y_train)
# result2=calculate_r2(X_test,y_test)